In [0]:
%tensorflow_version 1.x

from google.colab import drive
from google.colab.patches import cv2_imshow
drive.mount('/content/drive')


import os
os.chdir('/content/drive/My Drive/Colab Notebooks/TwoStageFramework')
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/TwoStageFramework')
import numpy as np
import tensorflow as tf
import glob
import copy
import cv2

#!wget https://github.com/wulfihm/rs_nn_training/raw/master/object_detector/working%20models/15022020_3x3Filter/frozen_inference_graph.pb

print(tf.__version__)

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = 'first_stage.pb'

IMAGE_SIZE_FIRSTSTAGE = (400, 300)


detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')



First Stage Inference

In [0]:
filelist = glob.glob('images/*.png')

cropped_images = []
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    for file in filelist:
        image = cv2.imread(file, cv2.IMREAD_UNCHANGED)
        image_res = cv2.resize(image, IMAGE_SIZE_FIRSTSTAGE)
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_res, axis=0)
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        # Actual detection.
        (boxes, scores, classes, num_detections) = sess.run(
            [boxes, scores, classes, num_detections],
            feed_dict={image_tensor: image_np_expanded})
        boxes_sq = np.squeeze(boxes)
        scores_sq = np.squeeze(scores)
        num_detections_sq = np.squeeze(num_detections)
        image_colored = copy.deepcopy(image)
        # Draw bounding boxes
        color = np.array([0, 255, 0], dtype=np.uint8)
        for i in range(0, int(num_detections_sq) - 1):
            if scores_sq[i] > 0.3:
                bbox = (int(boxes_sq[i, 0] * image.shape[0]), #xmin
                                int((boxes_sq[i, 2]) * image.shape[0]), #xmax
                                int(boxes_sq[i, 1] * image.shape[1]), #ymin
                                int((boxes_sq[i, 3]) * image.shape[1])) #ymax
                print("bbox dimensions: " + str(bbox[1]-bbox[0]) + "x" + str(bbox[3]-bbox[2]))
                image_colored[bbox[0], bbox[2]:bbox[3]] = color #obere kante
                image_colored[bbox[1], bbox[2]:bbox[3]] = color #unter kante
                image_colored[bbox[0]:bbox[1], bbox[2]] = color #linke kante
                image_colored[bbox[0]:bbox[1], bbox[3]] = color #rechte kante
                
                crop_img = image[bbox[0]:bbox[1], bbox[2]:bbox[3]]
                cropped_images.append(crop_img)
                cv2_imshow(crop_img)
        cv2_imshow(image_colored)


In [0]:
%load_ext tensorboard
%reload_ext tensorboard

%tensorboard --logdir "logs"